In [1]:
import numpy as np

from pad import pad
from new_fft import new_fft


# Fractional Fourier transform and its adjoint

Let $n \in \mathbb{N}$ and $c \in \mathbb{C}^n$. We denote by $n/2$ the quotient of the division of $n$ by $2$ (which corresponds to ``n//2`` in Python). We define:
$$
\mathcal{D}(n) = 
\begin{cases}
\llbracket - n/2, n/2 - 1\rrbracket &\text{ if } n \text{ is even} \\
\llbracket - n/2, n/2 \rrbracket &\text{ else}
\end{cases}
$$

Let $\alpha \in \mathbb{R}$. The $\alpha$-Fractional Fourier Transform of $c$ is defined as:
$$
\forall k \in \mathcal{D}(n), \, F^\alpha(c)(k) = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{\alpha k u}{n} \right)
$$

When $\alpha$ is rational, say $\alpha = \dfrac{a}{b}$, it becomes:
$$
\forall k \in \mathcal{D}(n), \, F^\alpha(c)(k) = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{a k u}{b n} \right)
$$

Using zero padding and the FFT, we can compute:
$$
\forall k' \in \mathcal{D}(nb), \, F^\alpha(c)(k') = \sum_{u \in \mathcal{D}(n)} c(u) \exp\left(-2i \pi \dfrac{k' u}{b n} \right)
$$

We then have to select the $k'$ values corresponding to the $a k$ ones.

In [2]:
def frac_fft(c: np.ndarray, a: int, b: int):
    """
    frac FFT with alpha = a / b
    """
    n = len(c)
    q_n, r_n = divmod(n, 2)
    m = b * len(c)
    q_m, r_m = divmod(m, 2)
    aux = pad(c, (m,))

    indices = (a * np.arange(-q_n, q_n + r_n)) % m  # between 0 and m - 1
    indices[indices >= q_m + r_m] -= m  # between -m//2 and -1

    indices += q_m

    return new_fft(aux)[indices]


In [3]:
def naive_frac_fourier(x, alpha):
    n = len(x)

    if n % 2 == 0:
        half_n = n//2
        # from -(n//2) to n//2 - 1 : 2*(n//2) = n points
        u = np.arange(-half_n, half_n)
        k = np.arange(-half_n, half_n)

    else:
        half_n = n//2
        # from -(n//2) to n//2 : 2*(n//2) + 1 = n points
        u = np.arange(-half_n, half_n + 1)
        k = np.arange(-half_n, half_n + 1)

    ku = np.exp(-2j * np.pi * alpha * np.einsum("k,u->ku", k, u) / n)
    res = np.einsum("u,ku->k", x, ku)

    return res


In [4]:
n_list = np.arange(10, 20)
a_list = np.arange(1, 7)
b_list = np.arange(2, 6)
l = []

for n in n_list:
    for a in a_list:
        for b in b_list:
            c = np.random.rand(n)
            l.append(np.max(np.abs(frac_fft(c, a, b) - naive_frac_fourier(c, a/b))))

print(np.max(l))


2.3658758876939235e-14


## Adjoint of ``frac_fft``

In [5]:
from frac_fft import frac_fft


In [6]:
def adj_frac_fft(c, a, b):
    return frac_fft(c, -a, b)


In [7]:
n_list = [4, 5]
a_list = np.arange(-10, 11)
b_list = [2, 3, 4]
err_list = []

for n in n_list:
    for a in a_list:
        for b in b_list:
            x = np.random.rand(n)
            y = np.random.rand(n)
            err_list.append(np.vdot(frac_fft(x, a, b), y) -
                            np.vdot(x, adj_frac_fft(y, a, b)))

print(np.max(np.abs(err_list)))


8.95090418262362e-16
